In [100]:
import random
from names import get_full_name
from tqdm import tqdm
from scipy import stats

class competition:
    
    def __init__(self, nr_players):
        self.player_poule = [player(skill_level = stats.truncnorm((0 - 0.5) / 0.15, (1 - 0.5) / 0.15, loc=0.5, scale=0.15).rvs(1)[0]) for i in tqdm(range(nr_players))]
        
    def play_rand_matches(self, nr_of_games):
        for game in tqdm(range(nr_of_games)):
            players = random.sample(range(0, len(self.player_poule)), 2)
            
            m = match(self.player_poule[players[0]], self.player_poule[players[1]])
            m.simulate_match()
        
    def show_standings(self):
        sorted_poule = sorted(self.player_poule, key=lambda x: x.rating, reverse=True)
        print("leaderboard standings")
        print("------------------------------------")
        for p in sorted_poule[0:10]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        for p in sorted_poule[-11:-1]:
            print("{} | {:4d} | {:.3f}".format(p.name.ljust(21), p.rating, p.skill_level))
        print("------------------------------------\n")
    

class match:
    
    def __init__(self, player1, player2, k_factor = 32):
        self.p1 = player1
        self.p2 = player2
        self.k_factor = k_factor
        
        self.es_p1 = 1 / (1 + 10**((player2.rating - player1.rating) / 400))
        self.es_p2 = 1 / (1 + 10**((player1.rating - player2.rating) / 400))
    
    def simulate_match(self):
        sl_p1 = self.p1.skill_level
        sl_p2 = self.p2.skill_level
        draw_odds = (1 - abs(sl_p1 - sl_p2)) * (sl_p1 + sl_p2)
        win_chances = [sl_p1, draw_odds, sl_p2]
        
        result_p1 = random.choices([1, 0.5, 0], weights = win_chances, k=1)[0]
        result_p2 = 1-result_p1
        
        self.p1.update_rating(self.k_factor, self.es_p1, result_p1)
        self.p2.update_rating(self.k_factor, self.es_p2, result_p2)
        
        return result_p1

class player:
    
    def __init__(self, rating = 1900, skill_level = 0.5, name = "filler name"):
        self.name = name
        self.rating = rating
        self.skill_level = skill_level
    
    def update_rating(self, k_factor, expected_score, result):
        change = k_factor * (result - expected_score)
        self.rating += round(change)


In [101]:
c = competition(2000)
c.show_standings()
c.play_rand_matches(1000000)
c.show_standings()

100%|██████████| 1000000/1000000 [00:09<00:00, 108833.58it/s]

leaderboard standings
------------------------------------
filler name           | 2042 | 0.501
filler name           | 2039 | 0.765
filler name           | 2038 | 0.576
filler name           | 2036 | 0.726
filler name           | 2032 | 0.721
filler name           | 2027 | 0.839
filler name           | 2026 | 0.684
filler name           | 2026 | 0.559
filler name           | 2026 | 0.646
filler name           | 2026 | 0.464
filler name           | 1729 | 0.108
filler name           | 1729 | 0.197
filler name           | 1726 | 0.207
filler name           | 1718 | 0.097
filler name           | 1699 | 0.053
filler name           | 1697 | 0.098
filler name           | 1695 | 0.057
filler name           | 1667 | 0.061
filler name           | 1640 | 0.032
filler name           | 1634 | 0.015
------------------------------------



In [89]:

#, name = get_full_name()
# gaat er vanuit dat iedereen tegen iedereen speelt
#Truncated normal distribution


import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from scipy import stats


X = 

X

0.4026514917292658

In [424]:
p1 = player(2000, 0.999)
p2 = player(1900, 0.0)
draw_odds = (1 - abs(p1.skill_level - p2.skill_level)) * (p1.skill_level + p2.skill_level)
print("ratings: {}    -    {}".format(p1.rating, p2.rating))
print("skill:   {:.2f} - {:.2f} - {:.2f}".format(p1.skill_level, draw_odds, p2.skill_level))

for i in range(100):
    m = match(p1, p2, 32)
    result = m.simulate_match()
    if i % 10 == 0:
        print("---------------------")
        print("p1 {}  -  {} p2".format(result, 1-result))
        print("ratings: {} - {}".format(p1.rating, p2.rating))
        

ratings: 2000    -    1900
skill:   1.00 - 0.00 - 0.00
---------------------
p1 1  -  0 p2
ratings: 2012 - 1888
---------------------
p1 1  -  0 p2
ratings: 2090 - 1810
---------------------
p1 1  -  0 p2
ratings: 2134 - 1766
---------------------
p1 1  -  0 p2
ratings: 2164 - 1736
---------------------
p1 1  -  0 p2
ratings: 2185 - 1715
---------------------
p1 1  -  0 p2
ratings: 2205 - 1695
---------------------
p1 1  -  0 p2
ratings: 2219 - 1681
---------------------
p1 1  -  0 p2
ratings: 2229 - 1671
---------------------
p1 1  -  0 p2
ratings: 2239 - 1661
---------------------
p1 1  -  0 p2
ratings: 2249 - 1651
